In [1]:
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.preprocessing import RobustScaler
import statistics
import math

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
def replace_with_median(df,col_name):
    for col_ind in range(0,df.shape[0]):
        prev_values = []
        for i in range(1,11):
            if str(df.iloc[col_ind][col_name+str(i)])=='nan':
                if prev_values:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = math.ceil(statistics.median(prev_values))
                else:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = 0
            else:
                prev_values.append(df.iloc[col_ind][col_name+str(i)])
    return df

In [3]:
def replace_with_median_fast(df,path):
    i=1
    nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
    for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = 0 
    for i in range(2,11):
        nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
        for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = math.ceil(statistics.median(df.loc[nan_i,[f'{path}{j}' for j in range(1,i)]]))
    return df

In [4]:
def replace_with_mean(df,col_name):
    for col_ind in range(0,df.shape[0]):
        prev_values = []
        for i in range(1,11):
            if str(df.iloc[col_ind][col_name+str(i)])=='nan':
                if prev_values:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = statistics.mean(prev_values)
                else:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = 0
            else:
                prev_values.append(df.iloc[col_ind][col_name+str(i)])
    return df

In [5]:
def replace_with_mean_fast(df,path):
    i=1
    nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
    for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = 0 
    for i in range(2,11):
        nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
        for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = statistics.mean(df.loc[nan_i,[f'{path}{j}' for j in range(1,i)]])
    return df

In [6]:
def replace_with_last_coach(df,col_name,isHome):
    for col_ind in range(0,df.shape[0]):
        prev_values = 0
        for i in range(1,11):
            if str(df.iloc[col_ind][col_name+str(i)])=='nan':
                if prev_values:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = prev_values
                else:
                    if isHome:
                        df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = df.iloc[col_ind]['home_team_coach_id']
                    else:
                        df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = df.iloc[col_ind]['away_team_coach_id']
            else:
                prev_values = df.iloc[col_ind][col_name+str(i)]
    return df

In [7]:
def replace_with_last_manager_fast(df,path,isHome):
    i=1
    nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
    for nan_i in nan_index: 
        if isHome: df.at[nan_i,f'{path}{i}'] = df.at[nan_i,'home_team_coach_id'] 
        else: df.at[nan_i,f'{path}{i}'] = df.at[nan_i,'away_team_coach_id'] 
    for i in range(2,11):
        nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
        for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = df.at[nan_i,f'{path}{i-1}']
    return df

In [8]:
def replace_with_last(df,col_name):
    for col_ind in range(0,df.shape[0]):
        prev_values = 0
        for i in range(1,11):
            if str(df.iloc[col_ind][col_name+str(i)])=='nan':
                if prev_values:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = prev_values
                else:
                    df.iat[col_ind,df.columns.values.tolist().index(col_name+str(i))] = df.iloc[col_ind]['league_id']
            else:
                prev_values = df.iloc[col_ind][col_name+str(i)]
    return df

In [9]:
def replace_with_last_fast(df,path):
    i=1
    nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
    for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = df.at[nan_i,'league_id'] 
    for i in range(2,11):
        nan_index = df.loc[pd.isna(df[f'{path}{i}']), :].index
        for nan_i in nan_index: df.at[nan_i,f'{path}{i}'] = df.at[nan_i,f'{path}{i-1}']
    return df

In [10]:

def add_features(df):
    for i in range(1, 11):
        df[f'home_team_history_match_DIFF_day_{i}'] = (df['match_date'] - df[f'home_team_history_match_date_{i}']).dt.days
        df[f'away_team_history_match_DIFF_day_{i}'] = (df['match_date'] - df[f'away_team_history_match_date_{i}']).dt.days
    
        df[f'home_team_history_DIFF_goal_{i}'] = df[f'home_team_history_goal_{i}'] - df[f'home_team_history_opponent_goal_{i}']
        df[f'away_team_history_DIFF_goal_{i}'] = df[f'away_team_history_goal_{i}'] - df[f'away_team_history_opponent_goal_{i}']
    
        df[f'home_team_result_{i}'] = np.where(df[f'home_team_history_DIFF_goal_{i}'] > 0, 2,
                         (np.where(df[f'home_team_history_DIFF_goal_{i}'] == 0, 1,
                                   np.where(df[f'home_team_history_DIFF_goal_{i}'].isna(), np.nan, 0))))
        df[f'away_team_result_{i}'] = np.where(df[f'away_team_history_DIFF_goal_{i}'] > 0, 2,
                         (np.where(df[f'away_team_history_DIFF_goal_{i}'] == 0, 1,
                                   np.where(df[f'away_team_history_DIFF_goal_{i}'].isna(), np.nan, 0))))
    
        # df[f'home_team_history_ELO_rating_{i}'] = 1/(1+10**((df[f'home_team_history_opponent_rating_{i}']-df[f'home_team_history_rating_{i}'])/10))
        # df[f'away_team_history_ELO_rating_{i}'] = 1/(1+10**((df[f'away_team_history_opponent_rating_{i}']-df[f'away_team_history_rating_{i}'])/10))
        # df[f'home_away_team_history_ELO_rating_{i}'] = 1/(1+10**((df[f'away_team_history_rating_{i}']-df[f'home_team_history_rating_{i}'])/10))

        df[f'home_team_history_diff_rating_{i}'] = df[f'home_team_history_rating_{i}'] - df[f'home_team_history_opponent_rating_{i}']
        df[f'away_team_history_diff_rating_{i}'] = df[f'away_team_history_rating_{i}'] - df[f'away_team_history_opponent_rating_{i}']
        df[f'home_away_team_history_diff_rating_{i}'] = df[f'home_team_history_rating_{i}'] - df[f'away_team_history_rating_{i}']
    
        df[f'home_team_history_SAME_coaX_{i}'] = np.where(df['home_team_coach_id']==df[f'home_team_history_coach_{i}'],1,0)
        df[f'away_team_history_SAME_coaX_{i}'] = np.where(df['away_team_coach_id']==df[f'away_team_history_coach_{i}'],1,0) 
    
        df[f'home_team_history_SAME_leaG_{i}'] = np.where(df['league_id']==df[f'home_team_history_league_id_{i}'],1,0)
        df[f'away_team_history_SAME_leaG_{i}'] = np.where(df['league_id']==df[f'away_team_history_league_id_{i}'],1,0) 
    
    return df

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\tz220\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
DEBUG = False

if DEBUG:
    train = train[:100]
    test = test[:100]

In [50]:
# print(f"Train: {train.shape}")
# train.head()
# ls = train.isna().sum() / train.shape[0]
# print(ls)
# ls.to_csv('nulls.csv')

id                                0.000000
target                            0.000000
home_team_name                    0.000009
away_team_name                    0.000009
match_date                        0.000000
                                    ...   
away_team_history_league_id_6     0.075952
away_team_history_league_id_7     0.088942
away_team_history_league_id_8     0.101814
away_team_history_league_id_9     0.115037
away_team_history_league_id_10    0.128144
Length: 190, dtype: float64


In [14]:
for col in train.filter(regex='date', axis=1).columns:
    train[col] = pd.to_datetime(train[col])
    test[col] = pd.to_datetime(test[col])

In [15]:
train_y = train['target'].copy()
train_x = train.drop(['target', 'home_team_name', 'away_team_name'], axis=1)

test_x = test.drop(['home_team_name', 'away_team_name'], axis=1)

In [16]:
mapping_dict = {1: 'H', 0: 'A'}

In [17]:
for i in range(1, 11):
    train_x[f'home_team_history_is_play_home_{i}'] = train_x[f'home_team_history_is_play_home_{i}'].map(mapping_dict)
    train_x[f'away_team_history_is_play_home_{i}'] = train_x[f'away_team_history_is_play_home_{i}'].map(mapping_dict)
    
    test_x[f'home_team_history_is_play_home_{i}'] = test_x[f'home_team_history_is_play_home_{i}'].map(mapping_dict)
    test_x[f'away_team_history_is_play_home_{i}'] = test_x[f'away_team_history_is_play_home_{i}'].map(mapping_dict)

In [18]:
cols = [f'home_team_history_is_play_home_{i}' for i in range(1, 11)]
train_x = pd.get_dummies(train_x, columns=cols)
cols = [f'away_team_history_is_play_home_{i}' for i in range(1, 11)]
train_x = pd.get_dummies(train_x, columns=cols)

cols = [f'home_team_history_is_play_home_{i}' for i in range(1, 11)]
test_x = pd.get_dummies(test_x, columns=cols)
cols = [f'away_team_history_is_play_home_{i}' for i in range(1, 11)]
test_x = pd.get_dummies(test_x, columns=cols)

In [19]:
for i in range(1, 11):
    train_x[f'home_team_history_is_cup_{i}'].fillna(0,inplace=True)
    train_x[f'away_team_history_is_cup_{i}'].fillna(0,inplace=True)

    test_x[f'home_team_history_is_cup_{i}'].fillna(0,inplace=True)
    test_x[f'away_team_history_is_cup_{i}'].fillna(0,inplace=True)

In [20]:
paths = ['home_team_history_goal_','home_team_history_opponent_goal_','away_team_history_goal_','away_team_history_opponent_goal_']
for path in paths:
    train_x = replace_with_median_fast(train_x,path=path)
    test_x = replace_with_median_fast(test_x,path=path)

In [21]:
# train_x = replace_with_median(train_x,'home_team_history_goal_')
# test_x = replace_with_median(test_x,'home_team_history_opponent_goal_')
# test_x = replace_with_median(test_x,'away_team_history_goal_')
# test_x = replace_with_median(test_x,'away_team_history_opponent_goal_')

# test_x = replace_with_median(test_x,'home_team_history_goal_')
# test_x = replace_with_median(test_x,'home_team_history_opponent_goal_')
# test_x = replace_with_median(test_x,'away_team_history_goal_')
# test_x = replace_with_median(test_x,'away_team_history_opponent_goal_')

In [22]:
paths = ['home_team_history_rating_','home_team_history_opponent_rating_','away_team_history_rating_','away_team_history_opponent_rating_']
for path in paths:
    train_x = replace_with_mean_fast(train_x,path=path)
    test_x = replace_with_mean_fast(test_x,path=path)

In [23]:
# train_x = replace_with_mean(train_x,'home_team_history_rating_')
# train_x = replace_with_mean(train_x,'home_team_history_opponent_rating_')
# train_x = replace_with_mean(train_x,'away_team_history_rating_')
# train_x = replace_with_mean(train_x,'away_team_history_opponent_rating_')

# test_x = replace_with_mean(test_x,'home_team_history_rating_')
# test_x = replace_with_mean(test_x,'home_team_history_opponent_rating_')
# test_x = replace_with_mean(test_x,'away_team_history_rating_')
# test_x = replace_with_mean(test_x,'away_team_history_opponent_rating_')

In [24]:
paths = ['home_team_history_coach_','away_team_history_coach_']
train_x = replace_with_last_manager_fast(train_x,path=paths[0],isHome=True)
train_x = replace_with_last_manager_fast(train_x,path=paths[1],isHome=False)

test_x = replace_with_last_manager_fast(test_x,path=paths[0],isHome=True)
test_x = replace_with_last_manager_fast(test_x,path=paths[1],isHome=False)

In [25]:
# train_x = replace_with_last_coach(train_x,'home_team_history_coach_',True)
# train_x = replace_with_last_coach(train_x,'away_team_history_coach_',False)

# test_x = replace_with_last_coach(test_x,'home_team_history_coach_',True)
# test_x = replace_with_last_coach(test_x,'away_team_history_coach_',False)

In [26]:
# train_x = replace_with_last(train_x,'home_team_history_league_id_')
# train_x = replace_with_last(train_x,'away_team_history_league_id_')

# test_x = replace_with_last(test_x,'home_team_history_league_id_')
# test_x = replace_with_last(test_x,'away_team_history_league_id_')

In [27]:
train_x = add_features(train_x)
test_x = add_features(test_x)

In [28]:
train_x.drop(train.filter(regex='date').columns, axis=1, inplace = True)
train_x.drop(train.filter(regex='league').columns, axis=1, inplace = True)
train_x.drop(train.filter(regex='coach').columns, axis=1, inplace = True)

test_x.drop(test.filter(regex='date').columns, axis=1, inplace = True)
test_x.drop(test.filter(regex='league').columns, axis=1, inplace = True)
test_x.drop(test.filter(regex='coach').columns, axis=1, inplace = True)

In [29]:
train_x['home_total_goals_scored'] = train_x.loc[:,[f'home_team_history_goal_{i}' for i in range(1,11)]].sum(axis=1)
train_x['home_total_goals_consumed'] = train_x.loc[:,[f'home_team_history_opponent_goal_{i}' for i in range(1,11)]].sum(axis=1)

train_x['away_total_goals_scored'] = train_x.loc[:,[f'away_team_history_goal_{i}' for i in range(1,11)]].sum(axis=1)
train_x['away_total_goals_consumed'] = train_x.loc[:,[f'away_team_history_opponent_goal_{i}' for i in range(1,11)]].sum(axis=1)

In [30]:
test_x['home_total_goals_scored'] = test_x.loc[:,[f'home_team_history_goal_{i}' for i in range(1,11)]].sum(axis=1)
test_x['home_total_goals_consumed'] = test_x.loc[:,[f'home_team_history_opponent_goal_{i}' for i in range(1,11)]].sum(axis=1)

test_x['away_total_goals_scored'] = test_x.loc[:,[f'away_team_history_goal_{i}' for i in range(1,11)]].sum(axis=1)
test_x['away_total_goals_consumed'] = test_x.loc[:,[f'away_team_history_opponent_goal_{i}' for i in range(1,11)]].sum(axis=1)

In [31]:
train_x['home_team_goals_scored_in_home'] = np.nan
train_x['home_team_goals_scored_in_away'] = np.nan
train_x['home_team_goals_consumed_in_home'] = np.nan
train_x['home_team_goals_consumed_in_away'] = np.nan
for index in train_x.index:
    train_x.at[index, 'home_team_goals_scored_in_home'] = train_x.loc[index:index,[f'home_team_history_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    train_x.at[index, 'home_team_goals_scored_in_away'] = train_x.loc[index:index,[f'home_team_history_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)
    train_x.at[index, 'home_team_goals_consumed_in_home'] = train_x.loc[index:index,[f'home_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    train_x.at[index, 'home_team_goals_consumed_in_away'] = train_x.loc[index:index,[f'home_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)

train_x['away_team_goals_scored_in_home'] = np.nan
train_x['away_team_goals_scored_in_away'] = np.nan
train_x['away_team_goals_consumed_in_home'] = np.nan
train_x['away_team_goals_consumed_in_away'] = np.nan
for index in train_x.index:
    train_x.at[index, 'away_team_goals_scored_in_home'] = train_x.loc[index:index,[f'away_team_history_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    train_x.at[index, 'away_team_goals_scored_in_away'] = train_x.loc[index:index,[f'away_team_history_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)
    train_x.at[index, 'away_team_goals_consumed_in_home'] = train_x.loc[index:index,[f'away_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    train_x.at[index, 'away_team_goals_consumed_in_away'] = train_x.loc[index:index,[f'away_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)

test_x['home_team_goals_scored_in_home'] = np.nan
test_x['home_team_goals_scored_in_away'] = np.nan
test_x['home_team_goals_consumed_in_home'] = np.nan
test_x['home_team_goals_consumed_in_away'] = np.nan
for index in test.index:
    test_x.at[index, 'home_team_goals_scored_in_home'] = test_x.loc[index:index,[f'home_team_history_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    test_x.at[index, 'home_team_goals_scored_in_away'] = test_x.loc[index:index,[f'home_team_history_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)
    test_x.at[index, 'home_team_goals_consumed_in_home'] = test_x.loc[index:index,[f'home_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    test_x.at[index, 'home_team_goals_consumed_in_away'] = test_x.loc[index:index,[f'home_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)

test_x['away_team_goals_scored_in_home'] = np.nan
test_x['away_team_goals_scored_in_away'] = np.nan
test_x['away_team_goals_consumed_in_home'] = np.nan
test_x['away_team_goals_consumed_in_away'] = np.nan
for index in test_x.index:
    test_x.at[index, 'away_team_goals_scored_in_home'] = test_x.loc[index:index,[f'away_team_history_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    test_x.at[index, 'away_team_goals_scored_in_away'] = test_x.loc[index:index,[f'away_team_history_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)
    test_x.at[index, 'away_team_goals_consumed_in_home'] = test_x.loc[index:index,[f'away_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].sum(axis=1)
    test_x.at[index, 'away_team_goals_consumed_in_away'] = test_x.loc[index:index,[f'away_team_history_opponent_goal_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].sum(axis=1)

In [32]:
train_x['home_current_manager_matches'] = train_x.loc[:,[f'home_team_history_SAME_coaX_{i}' for i in range(1,11)]].sum(axis=1)
train_x['away_current_manager_matches'] = train_x.loc[:,[f'away_team_history_SAME_coaX_{i}' for i in range(1,11)]].sum(axis=1)

In [33]:
test_x['home_current_manager_matches'] = test_x.loc[:,[f'home_team_history_SAME_coaX_{i}' for i in range(1,11)]].sum(axis=1)
test_x['away_current_manager_matches'] = test_x.loc[:,[f'away_team_history_SAME_coaX_{i}' for i in range(1,11)]].sum(axis=1)

In [34]:
train_x['home_average_rating_in_home'] = np.nan
train_x['home_average_rating_in_away'] = np.nan
for index in train_x.index:
    train_x.at[index, 'home_average_rating_in_home'] = train_x.loc[index:index,[f'home_team_history_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    train_x.at[index, 'home_average_rating_in_away'] = train_x.loc[index:index,[f'home_team_history_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

train_x['away_average_rating_in_home'] = np.nan
train_x['away_average_rating_in_away'] = np.nan
for index in train_x.index:
    train_x.at[index, 'away_average_rating_in_home'] = train_x.loc[index:index,[f'away_team_history_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    train_x.at[index, 'away_average_rating_in_away'] = train_x.loc[index:index,[f'away_team_history_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

In [35]:
test_x['home_average_rating_in_home'] = np.nan
test_x['home_average_rating_in_away'] = np.nan
for index in test_x.index:
    test_x.at[index, 'home_average_rating_in_home'] = test_x.loc[index:index,[f'home_team_history_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    test_x.at[index, 'home_average_rating_in_away'] = test_x.loc[index:index,[f'home_team_history_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

test_x['away_average_rating_in_home'] = np.nan
test_x['away_average_rating_in_away'] = np.nan
for index in test_x.index:
    test_x.at[index, 'away_average_rating_in_home'] = test_x.loc[index:index,[f'away_team_history_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    test_x.at[index, 'away_average_rating_in_away'] = test_x.loc[index:index,[f'away_team_history_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

In [36]:
# li = len([x for x in [i for i in range(1,11) if train_x.at[0,f'home_team_history_is_play_home_{i}_A']==1] if train_x.at[0,f'home_team_result_{x}']==2])
# print(li)
train_x['home_wins_in_home'] = np.nan
train_x['home_wins_in_away'] = np.nan
train_x['home_draws_in_home'] = np.nan
train_x['home_draws_in_away'] = np.nan
train_x['home_losses_in_home'] = np.nan
train_x['home_losses_in_away'] = np.nan
train_x['home_recorded_matches'] = np.nan
train_x['home_unrecorded_matches'] = np.nan
for index in train_x.index:
    train_x.at[index, 'home_wins_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'home_team_result_{x}']==2])
    train_x.at[index, 'home_wins_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'home_team_result_{x}']==2])
    
    train_x.at[index, 'home_draws_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'home_team_result_{x}']==1])
    train_x.at[index, 'home_draws_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'home_team_result_{x}']==1])
    
    train_x.at[index, 'home_losses_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'home_team_result_{x}']==0])
    train_x.at[index, 'home_losses_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'home_team_result_{x}']==0])

    train_x.at[index, 'home_recorded_matches'] = len([i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1 or train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1])
    train_x.at[index, 'home_unrecorded_matches'] = len([i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==0 and train_x.at[index,f'home_team_history_is_play_home_{i}_A']==0])


train_x['away_wins_in_home'] = np.nan
train_x['away_wins_in_away'] = np.nan
train_x['away_draws_in_home'] = np.nan
train_x['away_draws_in_away'] = np.nan
train_x['away_losses_in_home'] = np.nan
train_x['away_losses_in_away'] = np.nan
train_x['away_recorded_matches'] = np.nan
train_x['away_unrecorded_matches'] = np.nan
for index in train_x.index:
    train_x.at[index, 'away_wins_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'away_team_result_{x}']==2])
    train_x.at[index, 'away_wins_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'away_team_result_{x}']==2])
    
    train_x.at[index, 'away_draws_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'away_team_result_{x}']==1])
    train_x.at[index, 'away_draws_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'away_team_result_{x}']==1])
    
    train_x.at[index, 'away_losses_in_home'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if train_x.at[index,f'away_team_result_{x}']==0])
    train_x.at[index, 'away_losses_in_away'] = len([x for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if train_x.at[index,f'away_team_result_{x}']==0])

    train_x.at[index, 'away_recorded_matches'] = len([i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1 or train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1])
    train_x.at[index, 'away_unrecorded_matches'] = len([i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==0 and train_x.at[index,f'away_team_history_is_play_home_{i}_A']==0])

In [37]:
test_x['home_wins_in_home'] = np.nan
test_x['home_wins_in_away'] = np.nan
test_x['home_draws_in_home'] = np.nan
test_x['home_draws_in_away'] = np.nan
test_x['home_losses_in_home'] = np.nan
test_x['home_losses_in_away'] = np.nan
test_x['home_recorded_matches'] = np.nan
test_x['home_unrecorded_matches'] = np.nan
for index in test_x.index:
    test_x.at[index, 'home_wins_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'home_team_result_{x}']==2])
    test_x.at[index, 'home_wins_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'home_team_result_{x}']==2])
    
    test_x.at[index, 'home_draws_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'home_team_result_{x}']==1])
    test_x.at[index, 'home_draws_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'home_team_result_{x}']==1])
    
    test_x.at[index, 'home_losses_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'home_team_result_{x}']==0])
    test_x.at[index, 'home_losses_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'home_team_result_{x}']==0])

    test_x.at[index, 'home_recorded_matches'] = len([i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1 or test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1])
    test_x.at[index, 'home_unrecorded_matches'] = len([i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==0 and test_x.at[index,f'home_team_history_is_play_home_{i}_A']==0])


test_x['away_wins_in_home'] = np.nan
test_x['away_wins_in_away'] = np.nan
test_x['away_draws_in_home'] = np.nan
test_x['away_draws_in_away'] = np.nan
test_x['away_losses_in_home'] = np.nan
test_x['away_losses_in_away'] = np.nan
test_x['away_recorded_matches'] = np.nan
test_x['away_unrecorded_matches'] = np.nan
for index in test_x.index:
    test_x.at[index, 'away_wins_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'away_team_result_{x}']==2])
    test_x.at[index, 'away_wins_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'away_team_result_{x}']==2])
    
    test_x.at[index, 'away_draws_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'away_team_result_{x}']==1])
    test_x.at[index, 'away_draws_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'away_team_result_{x}']==1])
    
    test_x.at[index, 'away_losses_in_home'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1] if test_x.at[index,f'away_team_result_{x}']==0])
    test_x.at[index, 'away_losses_in_away'] = len([x for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1] if test_x.at[index,f'away_team_result_{x}']==0])

    test_x.at[index, 'away_recorded_matches'] = len([i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1 or test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1])
    test_x.at[index, 'away_unrecorded_matches'] = len([i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==0 and test_x.at[index,f'away_team_history_is_play_home_{i}_A']==0])

In [38]:
train_x['home_team_average_rest'] = train_x['home_team_history_match_DIFF_day_10'] / 10
train_x['away_team_average_rest'] = train_x['away_team_history_match_DIFF_day_10'] / 10

test_x['home_team_average_rest'] = test_x['home_team_history_match_DIFF_day_10'] / 10
test_x['away_team_average_rest'] = test_x['away_team_history_match_DIFF_day_10'] / 10

In [39]:
# print(train_x.columns.values)

In [40]:
train_x['home_team_history_current_rest'] = train_x['home_team_history_match_DIFF_day_1']
test_x['home_team_history_current_rest'] = test_x['home_team_history_match_DIFF_day_1']

In [41]:
train_x['home_average_rating_difference_in_home'] = np.nan
train_x['home_average_rating_difference_in_away'] = np.nan
train_x['away_average_rating_difference_in_home'] = np.nan
train_x['away_average_rating_difference_in_away'] = np.nan
for index in train_x.index:
    train_x.at[index, 'home_average_rating_difference_in_home'] = train_x.loc[index:index,[f'home_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    train_x.at[index, 'home_average_rating_difference_in_away'] = train_x.loc[index:index,[f'home_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

    train_x.at[index, 'away_average_rating_difference_in_home'] = train_x.loc[index:index,[f'away_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    train_x.at[index, 'away_average_rating_difference_in_away'] = train_x.loc[index:index,[f'away_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if train_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

test_x['home_average_rating_difference_in_home'] = np.nan
test_x['home_average_rating_difference_in_away'] = np.nan
test_x['away_average_rating_difference_in_home'] = np.nan
test_x['away_average_rating_difference_in_away'] = np.nan
for index in test_x.index:
    test_x.at[index, 'home_average_rating_difference_in_home'] = test_x.loc[index:index,[f'home_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    test_x.at[index, 'home_average_rating_difference_in_away'] = test_x.loc[index:index,[f'home_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'home_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)

    test_x.at[index, 'away_average_rating_difference_in_home'] = test_x.loc[index:index,[f'away_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_H']==1]]].mean(axis=1)
    test_x.at[index, 'away_average_rating_difference_in_away'] = test_x.loc[index:index,[f'away_team_history_diff_rating_{x}' for x in [i for i in range(1,11) if test_x.at[index,f'away_team_history_is_play_home_{i}_A']==1]]].mean(axis=1)


In [42]:
def dropCols(df):
    drop_list = [
        'home_team_history_is_cup_',
        'home_team_history_goal_',
        'home_team_history_opponent_goal_',
        'home_team_history_rating_',
        'home_team_history_opponent_rating_',
        'away_team_history_is_cup_',
        'away_team_history_goal_',
        'away_team_history_opponent_goal_',
        'away_team_history_rating_',
        'away_team_history_opponent_rating_',
        'home_team_history_is_play_home_',
        'away_team_history_is_play_home_',
        'home_team_history_match_DIFF_day_',
        'away_team_history_match_DIFF_day_',
        'home_team_history_DIFF_goal_',
        'away_team_history_DIFF_goal_',
        'home_team_result_',
        'away_team_result_',
        'home_team_history_diff_rating_',
        'away_team_history_diff_rating_',
        'home_away_team_history_diff_rating_',
        'home_team_history_SAME_coaX_',
        'away_team_history_SAME_coaX_',
        'home_team_history_SAME_leaG_',
        'away_team_history_SAME_leaG_',
        'home_team_history_match_DIFF_day_'
    ]

    for pres in drop_list:
        # print(pres)
        df.drop(df.filter(regex=pres).columns, axis=1, inplace = True)

    return df



In [43]:
train_x.to_csv('train_with_all.csv')
test_x.to_csv('test_with_all.csv')

In [44]:
train_x = dropCols(train_x)
test_x = dropCols(test_x)

In [45]:
train_x.to_csv('train_reduced.csv')
test_x.to_csv('test_reduced.csv')


In [46]:
train_y.to_csv('target.csv',index=False)